In [2]:
import random
import torch
import sys
from ImportantConfig import Config
from sql2fea import TreeBuilder, value_extractor
from NET import TreeNet
from sql2fea import Sql2Vec
from TreeLSTM import SPINN
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as Data
from PGUtils import pgrunner
import numpy as np
import pandas as pd
from sql_feature.workload_embedder import PredicateEmbedderDoc2Vec

sys.path.append('/home/ubuntu/project/HyperQO')

config = Config()
# sys.stdout = open(config.log_file, "w")
random.seed(0)

if __name__ == "__main__":
    with open(config.queries_file) as f:
        import json

        queries = json.load(f)

    tree_builder = TreeBuilder()
    sql2vec = Sql2Vec()
    # 这里的 input_size 必须为偶数！
    value_network = SPINN(head_num=config.head_num, input_size=36, hidden_size=config.hidden_size, table_num=50,
                          sql_size=config.sql_size, attention_dim=30).to(config.device)
    for name, param in value_network.named_parameters():
        from torch.nn import init

        if len(param.shape) == 2:
            init.xavier_normal(param)
        else:
            init.uniform(param)

    treenet_model = TreeNet(tree_builder, value_network)

    mask = (torch.rand(1, config.head_num, device=config.device) < 0.9).long()

    train = pd.read_csv('./information/train.csv', index_col=0)
    queries = train['query'].values

    workload_embedder = PredicateEmbedderDoc2Vec(queries, 20, pgrunner)

    train.head()

/tmp/ipykernel_5117/3932973385.py:41: UserWarning: nn.init.uniform is now deprecated in favor of nn.init.uniform_.
  init.uniform(param)
/tmp/ipykernel_5117/3932973385.py:39: UserWarning: nn.init.xavier_normal is now deprecated in favor of nn.init.xavier_normal_.
  init.xavier_normal(param)
2024-02-29 15:43:12,644 : INFO : adding document #0 to Dictionary<0 unique tokens: []>
2024-02-29 15:43:12,929 : INFO : adding document #10000 to Dictionary<134 unique tokens: ['Index_Scan_customer_demographics_Filter_cd_gender=ANDcd_marital_status=ANDcd_education_status=Index_Cond_cd_demo_sk=store_sales.ss_cdemo_sk', 'Seq_Scan_date_dim_Filter_d_year=', 'Seq_Scan_promotion_Filter_p_channel_email=ORp_channel_event=', 'Index_Scan_customer_address_Filter_ca_country=ANDca_state=ANYORca_state=ANYORca_state=ANYANDca_state=ANYANDstore_sales.ss_net_profit>=ANDstore_sales.ss_net_profit<=ORca_state=ANYANDstore_sales.ss_net_profit>=ANDstore_sales.ss_net_profit<=ORca_state=ANYANDstore_sales.ss_net_profit>=ANDst

In [3]:
print(1)

1


In [ ]:
x = torch.tensor(train.index)
y = torch.tensor(train['cost_reduction_ratio'].values)

    # 定义损失函数和优化器
criterion = nn.MSELoss()  # 例如，均方误差损失
optimizer = treenet_model.optimizer  # 例如，Adam 优化器

Batch_Size = 32
torch_dataset = Data.TensorDataset(x, y)

    # 训练循环
run_cnt=1
for epoch in range(1):  # 例如，训练多个 epochs
    loader = Data.DataLoader(dataset=torch_dataset,
                                 batch_size=Batch_Size,
                                 shuffle=True)
    for batch_x, batch_y in loader:
        optimizer.zero_grad()  # 每个批次前先清零梯度
        batch_loss = 0
        for num in range(Batch_Size):
            sql = queries[batch_x[num]]
            target_value = batch_y[num]
            plan_json = pgrunner.getCostPlanJson(sql)
            sql_vec = workload_embedder.get_embedding([sql])

                # 计算损失
            loss, mean, variance, exp_variance = treenet_model.train(plan_json, sql_vec, target_value, mask,
                                                                         is_train=True)
            run_cnt+=1
            print("train loss, mean, variance, exp_variance : {} - {} - {} - {}".format(loss, mean, variance, exp_variance))
                # loss = treenet_model.optimize()

            batch_loss += loss  # 累积批次损失
            # print(type(batch_loss))
        # batch_loss.backward()
        # optimizer.step()
        
            # if run_cnt <1000 or run_cnt%10==0:
            #     loss_value, mean, _ = treenet_model.optimize()
            #     print("optimize loss, mean : {} = {}".format(loss_value, mean))
            #     if mean <= 0.1:
            #         break
        print("batch loss : {}".format(batch_loss / Batch_Size))

feature size : torch.Size([36])
feature size : torch.Size([36])
feature size : torch.Size([36])
feature size : torch.Size([36])
feature size : torch.Size([36])
feature size : torch.Size([36])
feature size : torch.Size([36])
feature size : torch.Size([36])
feature size : torch.Size([36])
feature size : torch.Size([36])
feature size : torch.Size([36])
feature size : torch.Size([36])
feature size : torch.Size([36])
feature size : torch.Size([36])
feature size : torch.Size([36])
feature size : torch.Size([36])
feature size : torch.Size([36])
feature size : torch.Size([36])
feature size : torch.Size([36])
feature size : torch.Size([36])
feature size : torch.Size([36])
len of plan_feature : torch.Size([1, 144])
train loss, mean, variance, exp_variance : 0.21124210953712463 - 0.6066794991493225 - 0.43400573059882436 - 1.0
feature size : torch.Size([36])
feature size : torch.Size([36])
feature size : torch.Size([36])
feature size : torch.Size([36])
feature size : torch.Size([36])
feature size 

In [ ]:
treenet_model.plan_to_value()